In [1]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import requests

In [2]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
spacex_df.dtypes

Unnamed: 0                    int64
Flight Number                 int64
Launch Site                  object
class                         int64
Payload Mass (kg)           float64
Booster Version              object
Booster Version Category     object
dtype: object

In [4]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center','color': '#503D36','font-size': 24}),
    html.Div([
        html.Label("Select Launch Site:"),
        dcc.Dropdown(id="site-dropdown",
            options=[
                 {'label': 'All Sites', 'value': 'ALL'},
                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                 {"label": "VAFB SLC-4E", "value": "VAFB SLC-4E"},
                 {"label": "KSC LC-39A", "value": "KSC LC-39A"},
                 {"label": "CCAFS SLC-40", "value": "CCAFS SLC-40"}
            ],
            value = "ALL",
            searchable = True
            ),
        dcc.Graph(id='success-pie-chart'),
        dcc.RangeSlider(
        id='payload-slider',
        min=spacex_df['Payload Mass (kg)'].min(),
        max=spacex_df['Payload Mass (kg)'].max(),
        step=1000,
        marks={0:"0", 100: "100"},
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]),
        dcc.Graph(id='success-payload-scatter-chart')
    ])
    
])

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value')]
)
def update_pie_chart(selected_sites):
    if not selected_sites or 'ALL' in selected_sites:  
        # If no site or 'ALL' is selected, use all rows in the dataframe
        fig = px.pie(spacex_df, names='class', title='Total Success Launches')
    else:
        
        if isinstance(selected_sites, str):
            selected_sites = [selected_sites]

        # Filter the dataframe to include only data for selected site(s)
        filtered_df = spacex_df[spacex_df['Launch Site'].isin(selected_sites)]
        
        # Create a pie chart for the selected site(s)
        fig = px.pie(filtered_df, names='class', title='Success and Failure for Selected Site(s)')

    return fig

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)
def update_scatter_chart(selected_sites, selected_payload_range):
    if not selected_sites or selected_sites == 'ALL':  
        # If no site or all sites are selected, use all rows in the dataframe
        filtered_df = spacex_df
    else:
        # Filter the dataframe to include only data for the selected site(s)
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_sites]

    # Filter the dataframe based on selected payload range
    filtered_df = filtered_df[
        (filtered_df['Payload Mass (kg)'] >= selected_payload_range[0]) &
        (filtered_df['Payload Mass (kg)'] <= selected_payload_range[1])
    ]

    # Create a scatter chart for the selected site(s)
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Payload Mass vs Class',
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)